## T3 - Manage Growth at SportStuff.com

In [1]:
import numpy as np
import pandas as pd
from pulp import *

In [2]:
# Specify the solver to be GLPK
#path_to = r'C:\Users\jzhangfv\Miniconda3\envs\env_tutorial\Library\bin\glpsol.exe'
path_to = r'D:\Miniconda\envs\env_tutorial\Library\bin\glpsol.exe'
solver = GLPK_CMD(path=path_to)

### Demonstration:

#### 1. Setup parameters

In [3]:
# Calculate demands for next three years: yearly increasing rate 0.8
demand = [[320, 200, 160, 220, 350, 175]]

for i in range(3):
    demand.append([val*(1+0.8)**(i+1) for val in demand[0]])

print("Demand matrix is:")
for i in range(4):
    print(f"Year {i+2007}: {demand[i]}")

Demand matrix is:
Year 2007: [320, 200, 160, 220, 350, 175]
Year 2008: [576.0, 360.0, 288.0, 396.0, 630.0, 315.0]
Year 2009: [1036.8000000000002, 648.0, 518.4000000000001, 712.8000000000001, 1134.0, 567.0]
Year 2010: [1866.2400000000002, 1166.4, 933.1200000000001, 1283.0400000000002, 2041.2000000000003, 1020.6000000000001]


In [4]:
# Calculate transportation cost by piece
loc = ['Seattle','Denver','St. Louis','Atlanta','Philadelphia']

trans_cost = [[2.00, 2.50, 3.50, 4.00, 5.00, 5.50],
              [2.50, 2.50, 2.50, 3.00, 4.00, 4.50],
              [3.50, 3.50, 2.50, 2.50, 3.00, 3.50],
              [4.00, 4.00, 3.00, 2.50, 3.00, 2.50],
              [4.50, 5.00, 3.00, 3.50, 2.50, 4.00]]

net_trans_cost = []
for i in range(5):
    net_trans_cost.append([(val-3)/4 for val in trans_cost[i]])


print("Net transportation cost matrix is:")
for i in range(5):
    print(f"{loc[i]}: {net_trans_cost[i]}")

Net transportation cost matrix is:
Seattle: [-0.25, -0.125, 0.125, 0.25, 0.5, 0.625]
Denver: [-0.125, -0.125, -0.125, 0.0, 0.25, 0.375]
St. Louis: [0.125, 0.125, -0.125, -0.125, 0.0, 0.125]
Atlanta: [0.25, 0.25, 0.0, -0.125, 0.0, -0.125]
Philadelphia: [0.375, 0.5, 0.0, 0.125, -0.125, 0.25]


#### 2. Setup decision variables

In [5]:
# X_ij denotes shipping quantity from location i to market j
n_loc = 5
n_market = 6

variable_names = [str(i)+str(j) for j in range(1, n_market+1) for i in range(1, n_loc+1)]
variable_names.sort()

X_variables = LpVariable.matrix("X", variable_names, lowBound= 0)
DV_allocation = np.array(X_variables).reshape(5,6)
print("Allocation Matrix:")
print(DV_allocation)

Allocation Matrix:
[[X_11 X_12 X_13 X_14 X_15 X_16]
 [X_21 X_22 X_23 X_24 X_25 X_26]
 [X_31 X_32 X_33 X_34 X_35 X_36]
 [X_41 X_42 X_43 X_44 X_45 X_46]
 [X_51 X_52 X_53 X_54 X_55 X_56]]


In [6]:
# Y_ij denotes whether a small store (j=1) or a large store (j=2) open at location i
variable_names = [str(i)+str(j) for j in [1,2] for i in range(1, n_loc+1)]
variable_names.sort()

Y_variables = LpVariable.matrix("Y", variable_names, cat = "Integer", lowBound= 0)
DV_openess = np.array(Y_variables).reshape(5,2)
print("Openess Matrix:")
print(DV_openess)

Openess Matrix:
[[Y_11 Y_12]
 [Y_21 Y_22]
 [Y_31 Y_32]
 [Y_41 Y_42]
 [Y_51 Y_52]]


#### 3. Objective function

In [7]:
# fixed cost: facility fix cost + inventory fix cost
fix_cost = [[300+475,500+475],
            [250+475,420+475],
            [220+475,375+475],
            [220+475,375+475],
            [240+475,400+475]]

lpSum(fix_cost*DV_openess)

775*Y_11 + 975*Y_12 + 725*Y_21 + 895*Y_22 + 695*Y_31 + 850*Y_32 + 695*Y_41 + 850*Y_42 + 715*Y_51 + 875*Y_52 + 0

In [8]:
# variable cost: facility varaible cost + inventory varaible cost
lpSum((0.2+0.165)*DV_allocation)

0.365*X_11 + 0.365*X_12 + 0.365*X_13 + 0.365*X_14 + 0.365*X_15 + 0.365*X_16 + 0.365*X_21 + 0.365*X_22 + 0.365*X_23 + 0.365*X_24 + 0.365*X_25 + 0.365*X_26 + 0.365*X_31 + 0.365*X_32 + 0.365*X_33 + 0.365*X_34 + 0.365*X_35 + 0.365*X_36 + 0.365*X_41 + 0.365*X_42 + 0.365*X_43 + 0.365*X_44 + 0.365*X_45 + 0.365*X_46 + 0.365*X_51 + 0.365*X_52 + 0.365*X_53 + 0.365*X_54 + 0.365*X_55 + 0.365*X_56 + 0.0

In [9]:
# transportation cost
print(lpSum(DV_allocation*net_trans_cost))

-0.25*X_11 - 0.125*X_12 + 0.125*X_13 + 0.25*X_14 + 0.5*X_15 + 0.625*X_16 - 0.125*X_21 - 0.125*X_22 - 0.125*X_23 + 0.25*X_25 + 0.375*X_26 + 0.125*X_31 + 0.125*X_32 - 0.125*X_33 - 0.125*X_34 + 0.125*X_36 + 0.25*X_41 + 0.25*X_42 - 0.125*X_44 - 0.125*X_46 + 0.375*X_51 + 0.5*X_52 + 0.125*X_54 - 0.125*X_55 + 0.25*X_56


In [10]:
model = LpProblem(name="sportstuff-problem", sense=LpMinimize)
model += lpSum(fix_cost*DV_openess) + lpSum((0.2+0.165)*DV_allocation) + lpSum(DV_allocation*net_trans_cost)

In [11]:
model.objective

0.11499999999999999*X_11 + 0.24*X_12 + 0.49*X_13 + 0.615*X_14 + 0.865*X_15 + 0.99*X_16 + 0.24*X_21 + 0.24*X_22 + 0.24*X_23 + 0.365*X_24 + 0.615*X_25 + 0.74*X_26 + 0.49*X_31 + 0.49*X_32 + 0.24*X_33 + 0.24*X_34 + 0.365*X_35 + 0.49*X_36 + 0.615*X_41 + 0.615*X_42 + 0.365*X_43 + 0.24*X_44 + 0.365*X_45 + 0.24*X_46 + 0.74*X_51 + 0.865*X_52 + 0.365*X_53 + 0.49*X_54 + 0.24*X_55 + 0.615*X_56 + 775*Y_11 + 975*Y_12 + 725*Y_21 + 895*Y_22 + 695*Y_31 + 850*Y_32 + 695*Y_41 + 850*Y_42 + 715*Y_51 + 875*Y_52 + 0.0

#### 4. Add constraints

In [12]:
# Openess constraint
for i in range(n_loc):
    print(lpSum(DV_openess[i][j] for j in range(2)) <= 1)
    model += lpSum(DV_openess[i][j] for j in range(2)) <= 1, "Openess const" + str(i)

Y_11 + Y_12 <= 1
Y_21 + Y_22 <= 1
Y_31 + Y_32 <= 1
Y_41 + Y_42 <= 1
Y_51 + Y_52 <= 1


In [13]:
# Demand constraint
for j in range(n_market):
    print(lpSum(DV_allocation[i][j] for i in range(n_loc)) >= demand[1][j])
    model += lpSum(DV_allocation[i][j] for i in range(n_loc)) >= demand[1][j], "Demand const" + str(j)

X_11 + X_21 + X_31 + X_41 + X_51 >= 576.0
X_12 + X_22 + X_32 + X_42 + X_52 >= 360.0
X_13 + X_23 + X_33 + X_43 + X_53 >= 288.0
X_14 + X_24 + X_34 + X_44 + X_54 >= 396.0
X_15 + X_25 + X_35 + X_45 + X_55 >= 630.0
X_16 + X_26 + X_36 + X_46 + X_56 >= 315.0


In [14]:
# Capacity constraint
for i in range(n_loc):
    print(lpSum(DV_allocation[i][j] for j in range(n_market)) <= 2000*DV_openess[i][0]+4000*DV_openess[i][1])
    model += lpSum(DV_allocation[i][j] for j in range(n_market)) <= 2000*DV_openess[i][0]+4000*DV_openess[i][1], "Capacity const" + str(i)

X_11 + X_12 + X_13 + X_14 + X_15 + X_16 - 2000*Y_11 - 4000*Y_12 <= 0
X_21 + X_22 + X_23 + X_24 + X_25 + X_26 - 2000*Y_21 - 4000*Y_22 <= 0
X_31 + X_32 + X_33 + X_34 + X_35 + X_36 - 2000*Y_31 - 4000*Y_32 <= 0
X_41 + X_42 + X_43 + X_44 + X_45 + X_46 - 2000*Y_41 - 4000*Y_42 <= 0
X_51 + X_52 + X_53 + X_54 + X_55 + X_56 - 2000*Y_51 - 4000*Y_52 <= 0


In [15]:
model.constraints

OrderedDict([('Openess_const0', 1*Y_11 + 1*Y_12 + -1 <= 0),
             ('Openess_const1', 1*Y_21 + 1*Y_22 + -1 <= 0),
             ('Openess_const2', 1*Y_31 + 1*Y_32 + -1 <= 0),
             ('Openess_const3', 1*Y_41 + 1*Y_42 + -1 <= 0),
             ('Openess_const4', 1*Y_51 + 1*Y_52 + -1 <= 0),
             ('Demand_const0',
              1*X_11 + 1*X_21 + 1*X_31 + 1*X_41 + 1*X_51 + -576.0 >= 0),
             ('Demand_const1',
              1*X_12 + 1*X_22 + 1*X_32 + 1*X_42 + 1*X_52 + -360.0 >= 0),
             ('Demand_const2',
              1*X_13 + 1*X_23 + 1*X_33 + 1*X_43 + 1*X_53 + -288.0 >= 0),
             ('Demand_const3',
              1*X_14 + 1*X_24 + 1*X_34 + 1*X_44 + 1*X_54 + -396.0 >= 0),
             ('Demand_const4',
              1*X_15 + 1*X_25 + 1*X_35 + 1*X_45 + 1*X_55 + -630.0 >= 0),
             ('Demand_const5',
              1*X_16 + 1*X_26 + 1*X_36 + 1*X_46 + 1*X_56 + -315.0 >= 0),
             ('Capacity_const0',
              1*X_11 + 1*X_12 + 1*X_13 + 1*

#### 5. Solve the model

In [16]:
model.solve(solver)

1

In [17]:
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"Minimal cost for 2008: {round(model.objective.value()*1000)}$.")

status: 1, Optimal
Minimal cost for 2008: 1857100$.


In [18]:
for var in model.variables():
     print(f"{var.name}: {var.value()}")

X_11: 0.0
X_12: 0.0
X_13: 0.0
X_14: 0.0
X_15: 0.0
X_16: 0.0
X_21: 0.0
X_22: 0.0
X_23: 0.0
X_24: 0.0
X_25: 0.0
X_26: 0.0
X_31: 576.0
X_32: 360.0
X_33: 288.0
X_34: 396.0
X_35: 630.0
X_36: 315.0
X_41: 0.0
X_42: 0.0
X_43: 0.0
X_44: 0.0
X_45: 0.0
X_46: 0.0
X_51: 0.0
X_52: 0.0
X_53: 0.0
X_54: 0.0
X_55: 0.0
X_56: 0.0
Y_11: 0
Y_12: 0
Y_21: 0
Y_22: 0
Y_31: 0
Y_32: 1
Y_41: 0
Y_42: 0
Y_51: 0
Y_52: 0


#### 6. How to better present the result?

In [19]:
for var in model.variables():
    if var.name[0] == "Y":
        loc_index = int(var.name[2])-1
        
        if round(var.value()) == 0: 
            continue
            
        if int(var.name[3]) == 1:
            size = "small"
        else:
            size = "large"
    
        print(f"{loc[loc_index]} opens {size} store.")

St. Louis opens large store.


In [20]:
for var in model.variables():
    if var.name[0] == "X":
        loc_index = int(var.name[2])-1
        
        if round(var.value()) == 0:
            continue
            
        place = var.name[3]
        print(f"{loc[loc_index]} delivers {round(var.value())} to market {place}")

St. Louis delivers 576 to market 1
St. Louis delivers 360 to market 2
St. Louis delivers 288 to market 3
St. Louis delivers 396 to market 4
St. Louis delivers 630 to market 5
St. Louis delivers 315 to market 6


### Exercise:

Use the model to answer following questions:
1. What are optimal configurations for year 2009 and 2010?
* 2009 minimal cost \$2,810,030. 
* 2010 minimal cost \$4,423,054.
2. What are optimal configuration if transportation costs are doubled?
* 2008 minimal cost \$3,774,225.
* 2009 minimal cost \$5,772,605. 
* 2010 minimal cost \$9,505,689.